In [2]:
import numpy as np
import tifffile as tiff
import matplotlib.pyplot as plt
from scipy import ndimage as ndi
from scipy.signal import find_peaks, savgol_filter
from skimage import measure, morphology, filters
import pandas as pd
import os

# ==============================
# CONFIGURATION
# ==============================
input_dir = "/Volumes/volume2/2025/D-Disease Modeling Team/E-PKP2/2025/00_ToAnalyze/stream/GFP/RK25DE14B-D21_GFP/22276/combined"
process_all = True      # 👈 Toggle: True = all combined TIFFs, False = only one
single_tif_name = "RK25DE14A-D28_Stream_C09_s1_t1_FITC_combined.tif"

frame_pad_left = 25
frame_pad_right = 10
std_thresh_factor = -0.5
min_size_px = 500
pixel_size_um = 3.14
frame_rate = 50.0
save_png = True          # 👈 Controls saving plots
show_plots = False       # 👈 Controls live plotting in notebook

# ==============================
# FUNCTIONS
# ==============================

def detect_events(data, smooth_win=15, prominence_factor=1.5, min_distance=5, baseline_drop=0.75):
    """
    Detect calcium peaks from global intensity trace.
    Filters out closely spaced 'double peaks' that do not return
    to at least 75% of the previous peak's height.
    """
    # --- Compute mean trace and smooth ---
    mean_trace = np.mean(data.reshape(data.shape[0], -1), axis=1)
    smoothed = savgol_filter(mean_trace, window_length=smooth_win, polyorder=3)
    std = np.std(smoothed)

    # --- Initial peak detection ---
    peaks, properties = find_peaks(smoothed, prominence=prominence_factor * std, distance=min_distance)

    # --- Merge double peaks (require baseline drop) ---
    filtered_peaks = []
    if len(peaks) > 0:
        filtered_peaks.append(peaks[0])
        for i in range(1, len(peaks)):
            prev = filtered_peaks[-1]
            curr = peaks[i]

            # local min between peaks
            valley = np.min(smoothed[prev:curr])
            prev_height = smoothed[prev]
            curr_height = smoothed[curr]

            # compute threshold (75% of smaller peak)
            threshold = baseline_drop * min(prev_height, curr_height)

            # accept only if valley < threshold (baseline recovered)
            if valley < threshold:
                filtered_peaks.append(curr)

    peaks = np.array(filtered_peaks, dtype=int)

    # --- Compute derivative for slope starts ---
    d1 = np.gradient(smoothed)
    eps = 0.1 * np.std(d1)
    persist = 3

    landmarks = []
    for p in peaks:
        left = None
        for i in range(p - 1, 0, -1):
            if np.all(d1[i:i + persist] > eps):
                left = i
                break
        if left is None:
            left = max(0, p - 10)
        landmarks.append({"left": int(left), "peak": int(p)})

    return landmarks, mean_trace, smoothed, peaks


def get_global_mask(data, min_size_px=500, relax=0.8):
    """Segment largest object in MIP once for all events."""
    mip = np.max(data, axis=0)
    mip_norm = (mip - np.min(mip)) / (np.max(mip) - np.min(mip) + 1e-6)
    thr = filters.threshold_otsu(mip_norm) * relax
    binary = mip_norm > thr
    filled = ndi.binary_fill_holes(binary)
    cleaned = morphology.remove_small_objects(filled, min_size=min_size_px)
    labels = measure.label(cleaned)
    props = measure.regionprops(labels)
    if not props:
        raise ValueError("❌ No objects found in segmentation.")
    largest_label = max(props, key=lambda x: x.area).label
    mask = ndi.binary_fill_holes(labels == largest_label)
    return mip_norm, mask


def analyze_event_std(event_data, global_mask, std_thresh_factor=0.0):
    """Compute dynamic pixel mask via per-pixel STD and half-max frame map."""
    pixel_std = np.std(event_data, axis=0)
    pixel_min = np.min(event_data, axis=0)
    pixel_max = np.max(event_data, axis=0)

    masked_std = pixel_std[global_mask]
    mean_std = np.mean(masked_std)
    std_std = np.std(masked_std)
    threshold = mean_std + std_thresh_factor * std_std
    dynamic_mask = global_mask & (pixel_std >= threshold)

    frame_map = np.full_like(pixel_max, np.nan, dtype=np.float32)
    valid_mask = dynamic_mask & (pixel_max > pixel_min + 1e-6)
    half_level = pixel_min + 0.75 * (pixel_max - pixel_min)
    for f in range(event_data.shape[0]):
        frame_slice = event_data[f]
        hit_mask = (frame_slice >= half_level) & valid_mask & np.isnan(frame_map)
        frame_map[hit_mask] = f
    return pixel_std, dynamic_mask, frame_map, threshold


def analyze_tif(tif_path, output_root):
    """Full pipeline for one TIFF file."""
    print(f"\n🔹 Analyzing {os.path.basename(tif_path)}")
    base_name = os.path.splitext(os.path.basename(tif_path))[0]
    out_dir = os.path.join(output_root, base_name)
    os.makedirs(out_dir, exist_ok=True)

    # --- Load data
    data = tiff.imread(tif_path).astype(np.float32)
    n_frames, H, W = data.shape

    # --- Detect events
    landmarks, mean_trace, smoothed, peaks = detect_events(data)
    if not landmarks:
        print("⚠️ No events detected — skipping file.")
        return []

    # --- Plot and save event trace
    plt.figure(figsize=(12, 4))
    plt.plot(mean_trace, alpha=0.4, label="Raw mean")
    plt.plot(smoothed, lw=2, label="Smoothed")
    plt.plot(peaks, smoothed[peaks], "o", label="Peaks")
    for lm in landmarks:
        plt.axvline(lm["left"], color="green", ls="--", alpha=0.6)
        plt.axvline(lm["peak"], color="red", ls="--", alpha=0.6)
    plt.legend(); plt.grid(alpha=0.3)
    plt.title(f"Detected Calcium Events — {base_name}")
    plt.xlabel("Frame"); plt.ylabel("Mean Intensity")

    trace_path = os.path.join(out_dir, f"{base_name}_event_trace.png")
    plt.tight_layout()
    plt.savefig(trace_path, dpi=200)
    if show_plots:
        plt.show()
    else:
        plt.close()
    print(f"📈 Saved trace plot → {trace_path}")

    # --- Global mask
    mip_norm, global_mask = get_global_mask(data, min_size_px=min_size_px)
    mask_file = os.path.join(out_dir, f"{base_name}_globalmask.tif")
    tiff.imwrite(mask_file, global_mask.astype(np.uint8))

    # --- Plot and save mask
    plt.figure(figsize=(6, 6))
    plt.imshow(mip_norm, cmap="gray")
    plt.imshow(global_mask, cmap="Reds", alpha=0.4)
    plt.title(f"Global Mask — {base_name}")
    plt.axis("off")
    plt.savefig(os.path.join(out_dir, f"{base_name}_mask_overlay.png"), dpi=200)
    plt.close()

    # --- Analyze each event
    event_results = []
    for i, ev in enumerate(landmarks, 1):
        start = max(ev["left"] - frame_pad_left, 0)
        end = min(ev["peak"] + frame_pad_right, n_frames)
        event_data = data[start:end]

        pixel_std, dynamic_mask, frame_map, threshold = analyze_event_std(event_data, global_mask, std_thresh_factor)
        valid_frames = frame_map[np.isfinite(frame_map)]
        if valid_frames.size > 0:
            rel_min, rel_max = int(np.nanmin(valid_frames)), int(np.nanmax(valid_frames))
            abs_min, abs_max = start + rel_min, start + rel_max
        else:
            rel_min = rel_max = abs_min = abs_max = np.nan

        # --- Propagation speed
        grad_y, grad_x = np.gradient(frame_map)
        grad_mag = np.sqrt(grad_x**2 + grad_y**2)
        with np.errstate(divide='ignore', invalid='ignore'):
            speed_map = (pixel_size_um / grad_mag) * frame_rate
            speed_map[~np.isfinite(speed_map)] = np.nan
        speed_map[~dynamic_mask] = np.nan

        mean_speed = np.nanmean(speed_map)
        median_speed = np.nanmedian(speed_map)
        std_speed = np.nanstd(speed_map)

        # --- Save event-level maps
        tiff.imwrite(os.path.join(out_dir, f"{base_name}_event{i}_frame_map.tif"),
                     np.nan_to_num(frame_map, nan=0).astype(np.float32))
        tiff.imwrite(os.path.join(out_dir, f"{base_name}_event{i}_speed_map.tif"),
                     np.nan_to_num(speed_map, nan=0).astype(np.float32))

        # --- Visualization per event
        fig, axes = plt.subplots(1, 5, figsize=(22, 5))
        axes[0].imshow(mip_norm, cmap="gray")
        axes[0].set_title("Global Max Intensity"); axes[0].axis("off")

        axes[1].imshow(pixel_std, cmap="magma")
        axes[1].imshow(global_mask, cmap="Greens", alpha=0.2)
        axes[1].set_title(f"STD (Event {i})"); axes[1].axis("off")

        axes[2].imshow(mip_norm, cmap="gray")
        axes[2].imshow(dynamic_mask, cmap="Blues", alpha=0.5)
        axes[2].set_title(f"Dynamic Pixels (STD ≥ {threshold:.2f})"); axes[2].axis("off")

        im3 = axes[3].imshow(frame_map + start, cmap="turbo")
        axes[3].set_title(f"Frame index (abs)"); axes[3].axis("off")
        fig.colorbar(im3, ax=axes[3], fraction=0.046, pad=0.04, label="Frame #")

        im4 = axes[4].imshow(speed_map, cmap="plasma")
        axes[4].set_title("Propagation Speed (µm/s)")
        axes[4].axis("off")
        fig.colorbar(im4, ax=axes[4], fraction=0.046, pad=0.04, label="Speed (µm/s)")

        plt.suptitle(f"{base_name} — Event {i} | Mean Speed = {mean_speed:.2f} µm/s", fontsize=14)
        plt.tight_layout()
        event_plot_path = os.path.join(out_dir, f"{base_name}_event{i}_summary.png")
        plt.savefig(event_plot_path, dpi=200)
        plt.close(fig)
        print(f"💾 Saved event plot → {event_plot_path}")

        event_results.append({
            "file": base_name,
            "event_id": i,
            "start_frame": start,
            "peak_frame": ev["peak"],
            "end_frame": end,
            "threshold": threshold,
            "dynamic_pixel_count": int(np.sum(dynamic_mask)),
            "frame_range_rel": (rel_min, rel_max),
            "frame_range_abs": (abs_min, abs_max),
            "mean_speed_um_s": mean_speed,
            "median_speed_um_s": median_speed,
            "std_speed_um_s": std_speed,
            "difference": rel_max - rel_min,
        })
    return event_results


# ==============================
# MAIN RUN
# ==============================
if process_all:
    tif_files = sorted([os.path.join(input_dir, f)
                        for f in os.listdir(input_dir)
                        if f.endswith("_combined.tif")])
else:
    tif_files = [os.path.join(input_dir, single_tif_name)]

all_results = []
for tif_path in tif_files:
    all_results.extend(analyze_tif(tif_path, input_dir))

# --- Export summary CSV ---
if all_results:
    df = pd.DataFrame(all_results)
    csv_path = os.path.join(input_dir, "CalciumEventSummary.csv")
    df.to_csv(csv_path, index=False)
    print(f"\n✅ Exported summary CSV: {csv_path}")
    print(df[["file", "event_id", "difference", "mean_speed_um_s"]])
else:
    print("\n⚠️ No events processed.")


🔹 Analyzing RK25DE14B-D21_Stream_C07_s1_t1_FITC_combined.tif
📈 Saved trace plot → /Volumes/volume2/2025/D-Disease Modeling Team/E-PKP2/2025/00_ToAnalyze/stream/GFP/RK25DE14B-D21_GFP/22276/combined/RK25DE14B-D21_Stream_C07_s1_t1_FITC_combined/RK25DE14B-D21_Stream_C07_s1_t1_FITC_combined_event_trace.png
💾 Saved event plot → /Volumes/volume2/2025/D-Disease Modeling Team/E-PKP2/2025/00_ToAnalyze/stream/GFP/RK25DE14B-D21_GFP/22276/combined/RK25DE14B-D21_Stream_C07_s1_t1_FITC_combined/RK25DE14B-D21_Stream_C07_s1_t1_FITC_combined_event1_summary.png
💾 Saved event plot → /Volumes/volume2/2025/D-Disease Modeling Team/E-PKP2/2025/00_ToAnalyze/stream/GFP/RK25DE14B-D21_GFP/22276/combined/RK25DE14B-D21_Stream_C07_s1_t1_FITC_combined/RK25DE14B-D21_Stream_C07_s1_t1_FITC_combined_event2_summary.png
💾 Saved event plot → /Volumes/volume2/2025/D-Disease Modeling Team/E-PKP2/2025/00_ToAnalyze/stream/GFP/RK25DE14B-D21_GFP/22276/combined/RK25DE14B-D21_Stream_C07_s1_t1_FITC_combined/RK25DE14B-D21_Stream_C07_

KeyboardInterrupt: 


🔹 Analyzing RK25DE14B-D21_Stream_C07_s1_t1_FITC_combined.tif
💾 Saved event plot → /Users/lokesh.pimpale/Desktop/work_dir/RK/untitled folder/combined/RK25DE14B-D21_Stream_C07_s1_t1_FITC_combined/RK25DE14B-D21_Stream_C07_s1_t1_FITC_combined_event1_summary.png
💾 Saved event plot → /Users/lokesh.pimpale/Desktop/work_dir/RK/untitled folder/combined/RK25DE14B-D21_Stream_C07_s1_t1_FITC_combined/RK25DE14B-D21_Stream_C07_s1_t1_FITC_combined_event2_summary.png
💾 Saved event plot → /Users/lokesh.pimpale/Desktop/work_dir/RK/untitled folder/combined/RK25DE14B-D21_Stream_C07_s1_t1_FITC_combined/RK25DE14B-D21_Stream_C07_s1_t1_FITC_combined_event3_summary.png

🔹 Analyzing RK25DE14B-D21_Stream_C09_s1_t1_FITC_combined.tif
💾 Saved event plot → /Users/lokesh.pimpale/Desktop/work_dir/RK/untitled folder/combined/RK25DE14B-D21_Stream_C09_s1_t1_FITC_combined/RK25DE14B-D21_Stream_C09_s1_t1_FITC_combined_event1_summary.png
💾 Saved event plot → /Users/lokesh.pimpale/Desktop/work_dir/RK/untitled folder/combined/


🔹 Analyzing RK25DE14B-D21_Stream_C07_s1_t1_FITC_combined.tif


/var/folders/t_/2bd73txx5q3c7fpsfpwxb0gm0000gn/T/ipykernel_81601/3158461891.py:197: RuntimeWarning: Mean of empty slice
  grad_mean = np.nanmean(grad_flat_mag[dynamic_mask])


🌀 Saved flow debug plot → /Users/lokesh.pimpale/Desktop/work_dir/RK/untitled folder/combined/RK25DE14B-D21_Stream_C07_s1_t1_FITC_combined/RK25DE14B-D21_Stream_C07_s1_t1_FITC_combined_event1_flow_debug.png
💾 Saved event plot → /Users/lokesh.pimpale/Desktop/work_dir/RK/untitled folder/combined/RK25DE14B-D21_Stream_C07_s1_t1_FITC_combined/RK25DE14B-D21_Stream_C07_s1_t1_FITC_combined_event1_summary.png
🌀 Saved flow debug plot → /Users/lokesh.pimpale/Desktop/work_dir/RK/untitled folder/combined/RK25DE14B-D21_Stream_C07_s1_t1_FITC_combined/RK25DE14B-D21_Stream_C07_s1_t1_FITC_combined_event2_flow_debug.png
💾 Saved event plot → /Users/lokesh.pimpale/Desktop/work_dir/RK/untitled folder/combined/RK25DE14B-D21_Stream_C07_s1_t1_FITC_combined/RK25DE14B-D21_Stream_C07_s1_t1_FITC_combined_event2_summary.png
🌀 Saved flow debug plot → /Users/lokesh.pimpale/Desktop/work_dir/RK/untitled folder/combined/RK25DE14B-D21_Stream_C07_s1_t1_FITC_combined/RK25DE14B-D21_Stream_C07_s1_t1_FITC_combined_event3_flow_d

In [4]:
import numpy as np
import tifffile as tiff
import matplotlib.pyplot as plt
from scipy import ndimage as ndi
from scipy.signal import find_peaks, savgol_filter
from skimage import measure, morphology, filters
import pandas as pd
import cv2
import os

# ==============================
# CONFIGURATION
# ==============================
input_dir = '/Users/lokesh.pimpale/Desktop/work_dir/RK/untitled folder/combined'
process_all = True
single_tif_name = "RK25DE14A-D28_Stream_C09_s1_t1_FITC_combined.tif"

frame_pad_left = 25
frame_pad_right = 10
std_thresh_factor = -0.5
min_size_px = 500
pixel_size_um = 3.14
frame_rate = 50.0
save_png = True
show_plots = False

arrow_scale = 1.0
arrow_width = 0.075
flow_bin_size = 50      # binning for local averaging

# ==============================
# FUNCTIONS
# ==============================

def detect_events(data, smooth_win=15, prominence_factor=1.5, min_distance=5, baseline_drop=0.75):
    """Detect calcium peaks from global intensity trace."""
    mean_trace = np.mean(data.reshape(data.shape[0], -1), axis=1)
    smoothed = savgol_filter(mean_trace, window_length=smooth_win, polyorder=3)
    std = np.std(smoothed)
    peaks, _ = find_peaks(smoothed, prominence=prominence_factor * std, distance=min_distance)

    filtered_peaks = []
    if len(peaks) > 0:
        filtered_peaks.append(peaks[0])
        for i in range(1, len(peaks)):
            prev, curr = filtered_peaks[-1], peaks[i]
            valley = np.min(smoothed[prev:curr])
            threshold = baseline_drop * min(smoothed[prev], smoothed[curr])
            if valley < threshold:
                filtered_peaks.append(curr)
    peaks = np.array(filtered_peaks, dtype=int)

    # Find left slope start
    d1 = np.gradient(smoothed)
    eps = 0.1 * np.std(d1)
    persist = 3
    landmarks = []
    for p in peaks:
        left = None
        for i in range(p - 1, 0, -1):
            if np.all(d1[i:i + persist] > eps):
                left = i
                break
        if left is None:
            left = max(0, p - 10)
        landmarks.append({"left": int(left), "peak": int(p)})
    return landmarks, mean_trace, smoothed, peaks


def get_global_mask(data, min_size_px=500, relax=0.8):
    """Segment largest object in MIP once for all events."""
    mip = np.max(data, axis=0)
    mip_norm = (mip - np.min(mip)) / (np.max(mip) - np.min(mip) + 1e-6)
    thr = filters.threshold_otsu(mip_norm) * relax
    binary = mip_norm > thr
    filled = ndi.binary_fill_holes(binary)
    cleaned = morphology.remove_small_objects(filled, min_size=min_size_px)
    labels = measure.label(cleaned)
    props = measure.regionprops(labels)
    if not props:
        raise ValueError("❌ No objects found in segmentation.")
    largest_label = max(props, key=lambda x: x.area).label
    mask = ndi.binary_fill_holes(labels == largest_label)
    return mip_norm, mask


def analyze_event_std(event_data, global_mask, std_thresh_factor=0.0):
    """Compute dynamic pixel mask via per-pixel STD and half-max frame map."""
    pixel_std = np.std(event_data, axis=0)
    pixel_min = np.min(event_data, axis=0)
    pixel_max = np.max(event_data, axis=0)
    masked_std = pixel_std[global_mask]
    mean_std = np.mean(masked_std)
    std_std = np.std(masked_std)
    threshold = mean_std + std_thresh_factor * std_std
    dynamic_mask = global_mask & (pixel_std >= threshold)
    frame_map = np.full_like(pixel_max, np.nan, dtype=np.float32)
    valid_mask = dynamic_mask & (pixel_max > pixel_min + 1e-6)
    half_level = pixel_min + 0.75 * (pixel_max - pixel_min)
    for f in range(event_data.shape[0]):
        frame_slice = event_data[f]
        hit_mask = (frame_slice >= half_level) & valid_mask & np.isnan(frame_map)
        frame_map[hit_mask] = f
    return pixel_std, dynamic_mask, frame_map, threshold


def compute_optical_flow_binned(event_data, mask, bin_size=10):
    """Compute optical flow and average into bins (PIV-like)."""
    n_frames, H, W = event_data.shape
    norm_data = (event_data - np.min(event_data)) / (np.ptp(event_data) + 1e-6)
    flow_sum = np.zeros((H, W, 2), dtype=np.float32)
    valid_count = np.zeros((H, W), dtype=np.int32)

    # Compute dense optical flow between consecutive frames
    for f in range(n_frames - 1):
        prev = (norm_data[f] * 255).astype(np.uint8)
        nxt = (norm_data[f + 1] * 255).astype(np.uint8)
        flow = cv2.calcOpticalFlowFarneback(
            prev, nxt, None,
            pyr_scale=0.5, levels=3, winsize=15,
            iterations=3, poly_n=5, poly_sigma=1.2, flags=0
        )
        flow_sum += flow
        valid_count += 1

    mean_flow = np.divide(flow_sum, valid_count[..., None], where=valid_count[..., None] > 0)
    flow_u, flow_v = mean_flow[..., 0], mean_flow[..., 1]
    flow_u[~mask] = np.nan
    flow_v[~mask] = np.nan

    # PIV-style bin averaging
    nY = H // bin_size
    nX = W // bin_size
    Xc, Yc, Uc, Vc, Mc = [], [], [], [], []
    for by in range(nY):
        for bx in range(nX):
            y0, y1 = by * bin_size, (by + 1) * bin_size
            x0, x1 = bx * bin_size, (bx + 1) * bin_size
            block_mask = mask[y0:y1, x0:x1]
            if not np.any(block_mask):
                continue
            u_block = flow_u[y0:y1, x0:x1][block_mask]
            v_block = flow_v[y0:y1, x0:x1][block_mask]
            if len(u_block) < 1:
                continue
            mean_u = np.nanmean(u_block)
            mean_v = np.nanmean(v_block)
            mag = np.sqrt(mean_u**2 + mean_v**2)
            Xc.append(x0 + bin_size / 2)
            Yc.append(y0 + bin_size / 2)
            Uc.append(mean_u)
            Vc.append(mean_v)
            Mc.append(mag)
    return np.array(Xc), np.array(Yc), np.array(Uc), np.array(Vc), np.array(Mc)


# ==============================
# MAIN ANALYSIS FUNCTION
# ==============================
def analyze_tif(tif_path, output_root):
    print(f"\n🔹 Analyzing {os.path.basename(tif_path)}")
    base_name = os.path.splitext(os.path.basename(tif_path))[0]
    out_dir = os.path.join(output_root, base_name)
    os.makedirs(out_dir, exist_ok=True)

    data = tiff.imread(tif_path).astype(np.float32)
    n_frames, H, W = data.shape

    landmarks, mean_trace, smoothed, peaks = detect_events(data)
    if not landmarks:
        print("⚠️ No events detected — skipping file.")
        return []

    mip_norm, global_mask = get_global_mask(data, min_size_px=min_size_px)
    event_results = []

    # Plot global trace
    plt.figure(figsize=(10, 4))
    plt.plot(smoothed, lw=2, label="Smoothed")
    plt.plot(peaks, smoothed[peaks], "ro", label="Detected Peaks")
    plt.legend()
    plt.xlabel("Frame")
    plt.ylabel("Mean Intensity")
    plt.title(f"Calcium Events — {base_name}")
    plt.tight_layout()
    plt.savefig(os.path.join(out_dir, f"{base_name}_event_trace.png"), dpi=200)
    plt.close()

    for i, ev in enumerate(landmarks, 1):
        start = max(ev["left"] - frame_pad_left, 0)
        end = min(ev["peak"] + frame_pad_right, n_frames)
        event_data = data[start:end]

        pixel_std, dynamic_mask, frame_map, threshold = analyze_event_std(event_data, global_mask, std_thresh_factor)

        # --- Propagation speed (frame gradient) ---
        grad_y, grad_x = np.gradient(frame_map)
        grad_mag = np.sqrt(grad_x**2 + grad_y**2)
        with np.errstate(divide='ignore', invalid='ignore'):
            speed_map = (pixel_size_um / grad_mag) * frame_rate
        speed_map[~np.isfinite(speed_map)] = np.nan
        speed_map[~dynamic_mask] = np.nan
        mean_speed = np.nanmean(speed_map)

        # --- Optical flow PIV-style ---
        Xq, Yq, Uq, Vq, Mq = compute_optical_flow_binned(event_data, dynamic_mask, bin_size=flow_bin_size)
        avg_velocity = np.nanmean(Mq) if len(Mq) > 0 else np.nan

        # --- PLOTS (retain all) ---
        fig, axes = plt.subplots(1, 6, figsize=(28, 6))
        axes[0].imshow(mip_norm, cmap="gray")
        axes[0].set_title("Global Max Intensity")
        axes[0].axis("off")

        axes[1].imshow(pixel_std, cmap="magma")
        axes[1].imshow(global_mask, cmap="Greens", alpha=0.3)
        axes[1].set_title(f"STD (Event {i})")
        axes[1].axis("off")

        axes[2].imshow(mip_norm, cmap="gray")
        axes[2].imshow(dynamic_mask, cmap="Blues", alpha=0.4)
        axes[2].set_title(f"Dynamic Mask (≥{threshold:.2f})")
        axes[2].axis("off")

        im3 = axes[3].imshow(frame_map + start, cmap="turbo")
        axes[3].set_title("Frame Map (abs)")
        axes[3].axis("off")
        fig.colorbar(im3, ax=axes[3], fraction=0.046, pad=0.04, label="Frame #")

        im4 = axes[4].imshow(speed_map, cmap="plasma")
        axes[4].set_title(f"Speed Map\nMean={mean_speed:.2f} µm/s")
        axes[4].axis("off")
        fig.colorbar(im4, ax=axes[4], fraction=0.046, pad=0.04, label="Speed (µm/s)")

        # --- Quiver panel ---
        axes[5].imshow(np.mean(event_data, axis=0), cmap="gray", alpha=0.6)
        if len(Xq) > 0 and np.nanmax(Mq) > 0.02:
            qv = axes[5].quiver(Xq, Yq, Uq, Vq, Mq,
                                cmap="viridis",
                                scale_units="xy",
                                scale=arrow_scale,
                                width=arrow_width,
                                headwidth=5,
                                headlength=7,
                                pivot="mid",
                                alpha=0.9)
            fig.colorbar(qv, ax=axes[5], fraction=0.046, pad=0.04, label="Velocity (a.u.)")
        else:
            axes[5].text(0.5, 0.5, "NO FLOW DETECTED", color="red",
                         fontsize=14, fontweight="bold", ha="center", va="center")
        axes[5].set_title(f"Optical Flow (bin={flow_bin_size})\nAvg Vel={avg_velocity:.3f}")
        axes[5].axis("off")

        plt.suptitle(f"{base_name} — Event {i}", fontsize=16)
        plt.tight_layout()
        outpath = os.path.join(out_dir, f"{base_name}_event{i}_summary_PIV.png")
        plt.savefig(outpath, dpi=200)
        plt.close()
        print(f"💾 Saved summary → {outpath}")

        event_results.append({
            "file": base_name,
            "event_id": i,
            "start_frame": start,
            "peak_frame": ev["peak"],
            "end_frame": end,
            "dynamic_pixel_count": int(np.sum(dynamic_mask)),
            "mean_speed_um_s": mean_speed,
            "avg_velocity_flow": avg_velocity,
            "flow_detected": len(Xq) > 0 and np.nanmax(Mq) > 0.02
        })

    return event_results


# ==============================
# MAIN RUN
# ==============================
if process_all:
    tif_files = sorted([os.path.join(input_dir, f)
                        for f in os.listdir(input_dir)
                        if f.endswith("_combined.tif")])
else:
    tif_files = [os.path.join(input_dir, single_tif_name)]

all_results = []
for tif_path in tif_files:
    all_results.extend(analyze_tif(tif_path, input_dir))

if all_results:
    df = pd.DataFrame(all_results)
    csv_path = os.path.join(input_dir, "CalciumEventSummary_PIVOpticalFlow.csv")
    df.to_csv(csv_path, index=False)
    print(f"\n✅ Exported summary CSV: {csv_path}")
    print(df[["file", "event_id", "mean_speed_um_s", "avg_velocity_flow", "flow_detected"]])
else:
    print("\n⚠️ No events processed.")


🔹 Analyzing RK25DE14B-D21_Stream_C07_s1_t1_FITC_combined.tif
💾 Saved summary → /Users/lokesh.pimpale/Desktop/work_dir/RK/untitled folder/combined/RK25DE14B-D21_Stream_C07_s1_t1_FITC_combined/RK25DE14B-D21_Stream_C07_s1_t1_FITC_combined_event1_summary_PIV.png
💾 Saved summary → /Users/lokesh.pimpale/Desktop/work_dir/RK/untitled folder/combined/RK25DE14B-D21_Stream_C07_s1_t1_FITC_combined/RK25DE14B-D21_Stream_C07_s1_t1_FITC_combined_event2_summary_PIV.png
💾 Saved summary → /Users/lokesh.pimpale/Desktop/work_dir/RK/untitled folder/combined/RK25DE14B-D21_Stream_C07_s1_t1_FITC_combined/RK25DE14B-D21_Stream_C07_s1_t1_FITC_combined_event3_summary_PIV.png

🔹 Analyzing RK25DE14B-D21_Stream_C09_s1_t1_FITC_combined.tif
💾 Saved summary → /Users/lokesh.pimpale/Desktop/work_dir/RK/untitled folder/combined/RK25DE14B-D21_Stream_C09_s1_t1_FITC_combined/RK25DE14B-D21_Stream_C09_s1_t1_FITC_combined_event1_summary_PIV.png
💾 Saved summary → /Users/lokesh.pimpale/Desktop/work_dir/RK/untitled folder/combined

In [9]:
import os
import numpy as np
import pandas as pd
import tifffile as tiff
import matplotlib.pyplot as plt
from scipy import ndimage as ndi
from scipy.signal import find_peaks, savgol_filter
from skimage import measure, morphology, filters


# =====================================================
# CONFIGURATION
# =====================================================
input_dir = '/Users/lokesh.pimpale/Desktop/work_dir/RK/untitled folder/combined'
process_all = True
single_tif_name = "RK25DE14A-D28_Stream_C09_s1_t1_FITC_combined.tif"

frame_pad_left  = 25
frame_pad_right = 10
std_thresh_factor = -0.5
min_size_px = 500

pixel_size_um = 3.14
frame_rate = 50.0
bin_size = 25  # spatial bin size for propagation calculation

save_png = True
show_plots = False


# =====================================================
# HELPER FUNCTIONS
# =====================================================
def normalize01(a):
    a = a.astype(np.float32)
    mn, mx = np.nanmin(a), np.nanmax(a)
    if not np.isfinite(mn) or not np.isfinite(mx) or mx - mn < 1e-9:
        return np.zeros_like(a, dtype=np.float32)
    return (a - mn) / (mx - mn)


def detect_events(data, smooth_win=7, prominence_factor=1.5, min_distance=5, baseline_drop=0.75):
    """Detect calcium peaks from global intensity trace."""
    mean_trace = np.mean(data.reshape(data.shape[0], -1), axis=1)
    smoothed = savgol_filter(mean_trace, window_length=min(smooth_win, len(mean_trace)//2*2+1), polyorder=3)
    std = np.std(smoothed)
    peaks, _ = find_peaks(smoothed, prominence=prominence_factor * std, distance=min_distance)

    filtered_peaks = []
    if len(peaks) > 0:
        filtered_peaks.append(peaks[0])
        for i in range(1, len(peaks)):
            prev, curr = filtered_peaks[-1], peaks[i]
            valley = np.min(smoothed[prev:curr])
            threshold = baseline_drop * min(smoothed[prev], smoothed[curr])
            if valley < threshold:
                filtered_peaks.append(curr)
    peaks = np.array(filtered_peaks, dtype=int)

    # Detect start of slope
    d1 = np.gradient(smoothed)
    eps = 0.1 * np.std(d1)
    persist = 3
    landmarks = []
    for p in peaks:
        left = None
        for i in range(p - 1, 0, -1):
            if np.all(d1[i:i + persist] > eps):
                left = i
                break
        if left is None:
            left = max(0, p - 10)
        landmarks.append({"left": int(left), "peak": int(p)})
    return landmarks, mean_trace, smoothed, peaks


def get_global_mask(data, min_size_px=500, relax=0.8):
    """Segment largest object from MIP."""
    mip = np.max(data, axis=0)
    mip_norm = normalize01(mip)
    thr = filters.threshold_otsu(mip_norm) * relax
    binary = mip_norm > thr
    filled = ndi.binary_fill_holes(binary)
    cleaned = morphology.remove_small_objects(filled, min_size=min_size_px)
    labels = measure.label(cleaned)
    props = measure.regionprops(labels)
    if not props:
        raise ValueError("❌ No objects found in segmentation.")
    largest_label = max(props, key=lambda x: x.area).label
    mask = ndi.binary_fill_holes(labels == largest_label)
    return mip_norm, mask


def analyze_event_std(event_data, global_mask, std_thresh_factor=0.0):
    """Compute dynamic pixel mask via per-pixel STD and half-max frame map."""
    pixel_std = np.std(event_data, axis=0)
    pixel_min = np.min(event_data, axis=0)
    pixel_max = np.max(event_data, axis=0)
    masked_std = pixel_std[global_mask]
    mean_std = np.mean(masked_std)
    std_std = np.std(masked_std)
    threshold = mean_std + std_thresh_factor * std_std
    dynamic_mask = global_mask & (pixel_std >= threshold)
    frame_map = np.full_like(pixel_max, np.nan, dtype=np.float32)
    valid_mask = dynamic_mask & (pixel_max > pixel_min + 1e-6)
    half_level = pixel_min + 0.75 * (pixel_max - pixel_min)
    for f in range(event_data.shape[0]):
        frame_slice = event_data[f]
        hit_mask = (frame_slice >= half_level) & valid_mask & np.isnan(frame_map)
        frame_map[hit_mask] = f
    return pixel_std, dynamic_mask, frame_map, threshold


def compute_propagation_distance(frame_map, dynamic_mask, pixel_size_um=3.14, frame_rate=50.0, bin_size=25):
    """
    Divide dynamic mask into bins. Find earliest and latest bins (first & last activation).
    Compute distance, time difference, and velocity.
    """
    H, W = frame_map.shape
    nY = H // bin_size
    nX = W // bin_size
    centers, times = [], []

    for by in range(nY):
        for bx in range(nX):
            y0, y1 = by * bin_size, (by + 1) * bin_size
            x0, x1 = bx * bin_size, (bx + 1) * bin_size
            block_mask = dynamic_mask[y0:y1, x0:x1]
            block_frames = frame_map[y0:y1, x0:x1][block_mask]
            block_frames = block_frames[np.isfinite(block_frames)]
            if block_frames.size < 3:
                continue
            centers.append((x0 + bin_size / 2, y0 + bin_size / 2))
            times.append(np.nanmean(block_frames))

    if not centers:
        return np.nan, np.nan, np.nan, None, None

    centers = np.array(centers)
    times = np.array(times)
    i_start = np.argmin(times)
    i_end = np.argmax(times)

    (x1, y1), (x2, y2) = centers[i_start], centers[i_end]
    frame_start, frame_end = times[i_start], times[i_end]
    px_dist = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    um_dist = px_dist * pixel_size_um
    time_s = (frame_end - frame_start) / frame_rate
    vel = um_dist / time_s if time_s > 0 else np.nan
    return um_dist, time_s, vel, (x1, y1), (x2, y2)


# =====================================================
# MAIN FUNCTION
# =====================================================
def analyze_tif(tif_path, output_root):
    print(f"\n🔹 Analyzing {os.path.basename(tif_path)}")
    base_name = os.path.splitext(os.path.basename(tif_path))[0]
    out_dir = os.path.join(output_root, base_name)
    os.makedirs(out_dir, exist_ok=True)

    # --- Load TIFF
    data = tiff.imread(tif_path).astype(np.float32)
    n_frames, H, W = data.shape

    # --- Detect events
    landmarks, mean_trace, smoothed, peaks = detect_events(data)
    if not landmarks:
        print("⚠️ No events detected — skipping file.")
        return []

    # --- Global mask
    mip_norm, global_mask = get_global_mask(data, min_size_px=min_size_px)

    # --- Plot mean trace
    plt.figure(figsize=(10, 4))
    plt.plot(smoothed, lw=2, label="Smoothed")
    plt.plot(peaks, smoothed[peaks], "ro", label="Detected Peaks")
    plt.legend()
    plt.xlabel("Frame")
    plt.ylabel("Mean Intensity")
    plt.title(f"Calcium Events — {base_name}")
    plt.tight_layout()
    plt.savefig(os.path.join(out_dir, f"{base_name}_event_trace.png"), dpi=200)
    plt.close()

    # --- Analyze each event
    event_results = []
    for i, ev in enumerate(landmarks, 1):
        start = max(ev["left"] - frame_pad_left, 0)
        end = min(ev["peak"] + frame_pad_right, n_frames)
        event_data = data[start:end]

        pixel_std, dynamic_mask, frame_map, threshold = analyze_event_std(event_data, global_mask, std_thresh_factor)

        # --- Propagation analysis
        dist_um, time_s, vel_um_s, start_pt, end_pt = compute_propagation_distance(
            frame_map, dynamic_mask, pixel_size_um=pixel_size_um, frame_rate=frame_rate, bin_size=bin_size
        )

        # --- Speed map for visual reference
        gy, gx = np.gradient(frame_map)
        grad_mag = np.sqrt(gx**2 + gy**2)
        with np.errstate(divide='ignore', invalid='ignore'):
            speed_map = (pixel_size_um / grad_mag) * frame_rate
        speed_map[~np.isfinite(speed_map)] = np.nan
        speed_map[~dynamic_mask] = np.nan
        mean_speed = np.nanmean(speed_map)

        # --- Plot figure
        fig, axes = plt.subplots(1, 6, figsize=(28, 6))
        axes[0].imshow(mip_norm, cmap="gray")
        axes[0].set_title("Global MIP")
        axes[0].axis("off")

        axes[1].imshow(pixel_std, cmap="magma")
        axes[1].imshow(global_mask, cmap="Greens", alpha=0.3)
        axes[1].set_title(f"STD (Event {i})")
        axes[1].axis("off")

        axes[2].imshow(mip_norm, cmap="gray")
        axes[2].imshow(dynamic_mask, cmap="Blues", alpha=0.4)
        axes[2].set_title(f"Dynamic Mask (≥{threshold:.2f})")
        axes[2].axis("off")

        im3 = axes[3].imshow(frame_map + start, cmap="turbo")
        axes[3].set_title("Frame Map (absolute)")
        axes[3].axis("off")
        fig.colorbar(im3, ax=axes[3], fraction=0.046, pad=0.04, label="Frame #")

        im4 = axes[4].imshow(speed_map, cmap="plasma")
        axes[4].set_title(f"Speed Map\nMean={mean_speed:.2f} µm/s")
        axes[4].axis("off")
        fig.colorbar(im4, ax=axes[4], fraction=0.046, pad=0.04, label="Speed (µm/s)")

        axes[5].imshow(np.mean(event_data, axis=0), cmap="gray", alpha=0.6)
        axes[5].set_title(f"Propagation bins ({bin_size}px)\nDist={dist_um:.1f}µm | Vel={vel_um_s:.1f}µm/s")
        axes[5].axis("off")

        if start_pt and end_pt:
            axes[5].scatter(*start_pt, s=160, c="lime", edgecolors="black", label="Start")
            axes[5].scatter(*end_pt, s=160, c="red", edgecolors="black", label="End")
            axes[5].plot([start_pt[0], end_pt[0]], [start_pt[1], end_pt[1]], color="white", lw=2, ls="--")
        axes[5].legend(loc="lower right", frameon=False)

        plt.suptitle(f"{base_name} — Event {i}", fontsize=15)
        plt.tight_layout()
        out_path = os.path.join(out_dir, f"{base_name}_event{i}_summary_propagation.png")
        plt.savefig(out_path, dpi=220)
        if show_plots: plt.show()
        plt.close(fig)
        print(f"💾 Saved → {out_path}")

        event_results.append({
            "file": base_name,
            "event_id": i,
            "start_frame": start,
            "peak_frame": ev["peak"],
            "end_frame": end,
            "dynamic_pixel_count": int(np.sum(dynamic_mask)),
            "mean_speed_um_s": mean_speed,
            "propagation_distance_um": dist_um,
            "propagation_time_s": time_s,
            "propagation_velocity_um_s": vel_um_s
        })
    return event_results


# =====================================================
# MAIN RUN
# =====================================================
if process_all:
    tif_files = sorted([
        os.path.join(input_dir, f) for f in os.listdir(input_dir)
        if f.endswith("_combined.tif")
    ])
else:
    tif_files = [os.path.join(input_dir, single_tif_name)]

all_results = []
for tif_path in tif_files:
    all_results.extend(analyze_tif(tif_path, input_dir))

if all_results:
    df = pd.DataFrame(all_results)
    csv_path = os.path.join(input_dir, "CalciumEventSummary_Propagation.csv")
    df.to_csv(csv_path, index=False)
    print(f"\n✅ Exported summary CSV: {csv_path}")
    print(df[["file", "event_id", "propagation_distance_um", "propagation_velocity_um_s"]])
else:
    print("\n⚠️ No events processed.")


🔹 Analyzing RK25DE14B-D21_Stream_C07_s1_t1_FITC_combined.tif
💾 Saved → /Users/lokesh.pimpale/Desktop/work_dir/RK/untitled folder/combined/RK25DE14B-D21_Stream_C07_s1_t1_FITC_combined/RK25DE14B-D21_Stream_C07_s1_t1_FITC_combined_event1_summary_propagation.png
💾 Saved → /Users/lokesh.pimpale/Desktop/work_dir/RK/untitled folder/combined/RK25DE14B-D21_Stream_C07_s1_t1_FITC_combined/RK25DE14B-D21_Stream_C07_s1_t1_FITC_combined_event2_summary_propagation.png
💾 Saved → /Users/lokesh.pimpale/Desktop/work_dir/RK/untitled folder/combined/RK25DE14B-D21_Stream_C07_s1_t1_FITC_combined/RK25DE14B-D21_Stream_C07_s1_t1_FITC_combined_event3_summary_propagation.png

🔹 Analyzing RK25DE14B-D21_Stream_C09_s1_t1_FITC_combined.tif
💾 Saved → /Users/lokesh.pimpale/Desktop/work_dir/RK/untitled folder/combined/RK25DE14B-D21_Stream_C09_s1_t1_FITC_combined/RK25DE14B-D21_Stream_C09_s1_t1_FITC_combined_event1_summary_propagation.png
💾 Saved → /Users/lokesh.pimpale/Desktop/work_dir/RK/untitled folder/combined/RK25DE1